#### Uniswap V3 Subgraph - Query Downloader

In [2]:
hex(11155111)

'0xaa36a7'

In [1]:
import requests
# pretty print is used to print the output in the console in an easy to read format
from pprint import pprint


# function to use requests.post to make an API call to the subgraph url
def run_query(q):

    # endpoint where you are making the request
    request = requests.post('https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'
                            '',
                            json={'query': q})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, q))


# The Graph query - Query Uniswap for a list of the top 10 pairs where the reserve is > 1000000 USD and the volume is >50000 USD
query = """

{
  tokenDayDatas(orderBy: date, orderDirection: asc,
    where: {
      token_: {symbol: "AAVE"}, date_gt: 1682333263
    }
  ) {
      date
      priceUSD
      token {
        name
        poolCount
        id
        symbol
        totalSupply
        totalValueLocked
        totalValueLockedUSD
        txCount
        volume
        volumeUSD
      }
    }
}
"""
result = run_query(query)

# print the results
print('Print Result - {}'.format(result))
print('#############')
# pretty print the results
pprint(result)

KeyboardInterrupt: 

In [23]:
from datetime import datetime
# converting timestamp to date
dt_object = datetime.fromtimestamp(result['data']['tokenDayDatas'][-1]['date'])

print('Recorded at:', dt_object)

Recorded at: 2021-08-11 01:00:00


In [22]:
print(datetime.fromtimestamp(1682467200))

2023-04-26 01:00:00


### Get names method

- Query the name, id, symbol of all tokens with significant usage on Uniswap (eg TVL > $1,000,000)

In [3]:
import requests
r = requests.get('https://gateway.ipfs.io/ipns/tokens.uniswap.org').json()

In [4]:
tokens = {}
for t in r['tokens']:
    if t['chainId'] == 1:
        tokens[t['symbol']] = t

In [5]:
len(tokens)

239

### Query a single token for all historical data

In [ ]:
    {{
    tokenDayDatas(
        first: 10
        orderBy: date
        orderDirection: asc
        where: {{date_gt: {last_date}, token_: {{symbol: "{symbol}"}}}}
    ) {{
        date
        priceUSD
        token {{
            name
            id
            symbol
            tradeVolume
            tradeVolumeUSD
            totalSupply
            txCount
            totalLiquidity
            }}
    }}
    }}

In [1]:
def build_query_v3(symbol, last_date=5):
    query = f"""
    {{
        tokenDayDatas(
            first: 1000
            orderBy: date
            orderDirection: asc
            where: {{
                token_: {{symbol: "{symbol}"}}
                date_gt: {last_date}
            }}
        ) {{
            date
            priceUSD
            token {{
                name
                poolCount
                id
                symbol
                totalSupply
                totalValueLocked
                totalValueLockedUSD
                txCount
                volume
                volumeUSD
            }}
        }}
    }}
    """
    return query

def build_query_v2(symbol, last_date=5):
    query = f"""
    {{
    tokenDayDatas(
        first: 10
        orderBy: date
        orderDirection: asc
        where: {{date_gt: {last_date}, token_: {{symbol: "{symbol}"}}}}
    ) {{
        date
        priceUSD
        token {{
            name
            id
            symbol
            tradeVolume
            tradeVolumeUSD
            totalSupply
            txCount
            totalLiquidity
            }}
        }}
    }}
    """
    return query


def run_query(q, version=3):
    '''function to use requests.post to make an API call to the subgraph url'''
    import requests
    # endpoint for request
    request = requests.post(f'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v{version}'
                            '',
                            json={'query': q})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, q))
    

def fetch_data(tokenData):
    '''Method for building query based on last available time'''
    import time

    for t in list(tokenData):

        # check if no data or if most recent value is older than 1 day
        last_date = 5 if not tokenData[t] else tokenData[t][-1]['date']
        if not tokenData[t] or (tokenData[t][-1]['date'] < (time.time() - (60 * 60 * 24))): 

            # build query with filter on date
            query_v3 = build_query_v3(t, last_date)
            try: 
                result = run_query(query_v3)['data']['tokenDayDatas']
                tokenData[t] = result if not tokenData[t] else tokenData[t].extend(result)
            except: 
                query_v2 = build_query_v2(t, last_date)
                try:
                    result = run_query(query_v2, version=2)['data']['tokenDayDatas']
                    tokenData[t] = result if not tokenData[t] else tokenData[t].extend(result)
                except:
                    print(query_v2)
            
        time.sleep(1)   # limit frequency of queries

    return  tokenData


In [8]:
query_v2 = build_query_v2('C98', last_date=5)
run_query(query_v2, version=2)['data']['tokenDayDatas']

[{'date': 1593993600,
  'priceUSD': '0.00009671648620782010681377098554565464',
  'token': {'name': 'C98 (coin98.net)',
   'id': '0x82619a31ab374f4b3d0ef8a74d8dc84d6a5c751d',
   'symbol': 'C98',
   'tradeVolume': '2658069.192703',
   'tradeVolumeUSD': '0',
   'totalSupply': '14336',
   'txCount': '13',
   'totalLiquidity': '0.000001'}},
 {'date': 1617667200,
  'priceUSD': '0',
  'token': {'name': 'C98 (coin98.net)',
   'id': '0x82619a31ab374f4b3d0ef8a74d8dc84d6a5c751d',
   'symbol': 'C98',
   'tradeVolume': '2658069.192703',
   'tradeVolumeUSD': '0',
   'totalSupply': '14336',
   'txCount': '13',
   'totalLiquidity': '0.000001'}},
 {'date': 1626307200,
  'priceUSD': '0.1066039522138705991586962741336059',
  'token': {'name': 'Coin98',
   'id': '0x277b5945076be047a1f652b954050a694f85b759',
   'symbol': 'C98',
   'tradeVolume': '902832.845096266067211359',
   'tradeVolumeUSD': '90738.25479533535127015760887457602',
   'totalSupply': '14336',
   'txCount': '58',
   'totalLiquidity': '0.00

In [249]:
query_v2 = build_query_v2(symbol=list(tokenData)[list(tokenData).index('C98')], last_date=5)

run_query(query_v2, version=2)['data']['tokenDayDatas']

[{'date': 1593993600,
  'priceUSD': '0.00009671648620782010681377098554565464',
  'token': {'name': 'C98 (coin98.net)',
   'id': '0x82619a31ab374f4b3d0ef8a74d8dc84d6a5c751d',
   'symbol': 'C98',
   'tradeVolume': '2658069.192703',
   'tradeVolumeUSD': '0',
   'totalSupply': '14336',
   'txCount': '13',
   'totalLiquidity': '0.000001'}},
 {'date': 1617667200,
  'priceUSD': '0',
  'token': {'name': 'C98 (coin98.net)',
   'id': '0x82619a31ab374f4b3d0ef8a74d8dc84d6a5c751d',
   'symbol': 'C98',
   'tradeVolume': '2658069.192703',
   'tradeVolumeUSD': '0',
   'totalSupply': '14336',
   'txCount': '13',
   'totalLiquidity': '0.000001'}},
 {'date': 1626307200,
  'priceUSD': '0.1066039522138705991586962741336059',
  'token': {'name': 'Coin98',
   'id': '0x277b5945076be047a1f652b954050a694f85b759',
   'symbol': 'C98',
   'tradeVolume': '902832.845096266067211359',
   'tradeVolumeUSD': '90738.25479533535127015760887457602',
   'totalSupply': '14336',
   'txCount': '58',
   'totalLiquidity': '0.00

In [262]:
test_tokenData = tokenData.copy()

In [264]:
import pandas as pd

names = []
values = []
for t in list(test_tokenData):
    names.append(t)
    length = 0 if not test_tokenData[t] else len(test_tokenData[t])
    values.append(length)

stats = {'symbol': names, 'observations':values}

In [266]:
value_totals = pd.DataFrame(stats)

In [268]:
value_totals['observations'].value_counts()

0      31
722    14
721     7
709     5
723     4
       ..
684     1
547     1
686     1
364     1
326     1
Name: observations, Length: 162, dtype: int64

#### Extract prices from JSON and add to CSV

In [274]:
import json
with open('data/tokenPriceData.json') as file:
    full_data = json.load(file) 

In [276]:
import pandas as pd
from datetime import datetime

# data = {
#     "1INCH": [
#         # ...
#     ],
#     # ...
# }

def unix_to_datetime(unix_time):
    return datetime.utcfromtimestamp(unix_time).strftime('%Y-%m-%d')

# Extract date and priceUSD from the data
extracted_data = [{'date': unix_to_datetime(item['date']), '1INCH': item['priceUSD']} for item in full_data['1INCH']]

# Create a pandas DataFrame
df = pd.DataFrame(extracted_data)

print(df)


           date                                 1INCH
0    2021-05-05   6.198889182909350886505987800422162
1    2021-05-06                                     0
2    2021-05-07                                     0
3    2021-05-08   7.501175367472296865262179965221419
4    2021-05-09   7.013277210932177249030120005274198
..          ...                                   ...
717  2023-04-22   0.499615126015460299085202419788568
718  2023-04-23  0.4920020605819506923451426390583383
719  2023-04-24  0.4887931085038128001571252921271883
720  2023-04-25  0.4882807108156256261262625488251618
721  2023-04-26  0.5014172123064452187510518521047217

[722 rows x 2 columns]


In [280]:
full_data.keys()

dict_keys(['1INCH', 'AAVE', 'ABT', 'ACH', 'ADX', 'AERGO', 'AGLD', 'AIOZ', 'ALCX', 'ALEPH', 'ALI', 'ALICE', 'ALPHA', 'AMP', 'ANKR', 'ANT', 'APE', 'API3', 'ARB', 'ARPA', 'ASH', 'ASM', 'AST', 'ATA', 'AUCTION', 'AUDIO', 'AVT', 'AXL', 'AXS', 'BADGER', 'BAL', 'BAND', 'BAT', 'BICO', 'BIT', 'BLUR', 'BLZ', 'BNT', 'BOBA', 'BOND', 'BTRST', 'BUSD', 'C98', 'cbETH', 'CELR', 'CHR', 'CHZ', 'CLV', 'COMP', 'COTI', 'COVAL', 'CQT', 'CRO', 'CRPT', 'CRV', 'CTSI', 'CTX', 'CUBE', 'CVC', 'CVX', 'DAI', 'DAR', 'DDX', 'DENT', 'DEXT', 'DIA', 'DNT', 'DPI', 'DREP', 'DYDX', 'DYP', 'ELA', 'ELON', 'ENJ', 'ENS', 'ERN', 'EUL', 'EUROC', 'FARM', 'FET', 'FIS', 'FORT', 'FORTH', 'FOX', 'FRAX', 'FTM', 'FX', 'FXS', 'GAL', 'GALA', 'GFI', 'GHST', 'GLM', 'GNO', 'GODS', 'GRT', 'GTC', 'GUSD', 'GYEN', 'HFT', 'HIGH', 'HOPR', 'IDEX', 'ILV', 'IMX', 'INDEX', 'INJ', 'INV', 'IOTX', 'JAM', 'JASMY', 'JUP', 'KEEP', 'KEY', 'KNC', 'KP3R', 'KRL', 'LCX', 'LDO', 'LINK', 'LOKA', 'LOOM', 'LPT', 'LQTY', 'LRC', 'LUSD', 'MANA', 'MASK', 'MATH', 'MATIC',

In [283]:
import pandas as pd
from datetime import datetime

def unix_to_datetime(unix_time):
    return datetime.utcfromtimestamp(unix_time).strftime('%Y-%m-%d')

# Extract date and priceUSD from the data for each key
extracted_data = []
for key in full_data.keys():
    if full_data[key]:
        for item in full_data[key]:
            try:
                entry = {'date': unix_to_datetime(item['date']), key: item['priceUSD']}
                extracted_data.append(entry)
            except:
                print(key)

# Create a pandas DataFrame
df = pd.DataFrame(extracted_data)

# Pivot the DataFrame so that each key becomes a column
df = df.pivot_table(index='date', columns=df.columns[:-1], values=df.columns[-1], aggfunc='first').reset_index()

df


ABT
ABT
ABT


ValueError: Grouper and axis must be same length

In [302]:
import pandas as pd
from datetime import datetime

def unix_to_datetime(unix_time):
    return datetime.utcfromtimestamp(unix_time).strftime('%Y-%m-%d')

dataframes = []

# Extract date and priceUSD from the data for each key and create a DataFrame for each key
for key in full_data.keys():
    if full_data[key]:
        try:
            extracted_data = [{'date': item['date'], key: item['priceUSD']} for item in full_data[key]]
            df_key = pd.DataFrame(extracted_data)
            dataframes.append(df_key)
        except:
            print(key)

# Merge all the DataFrames using the 'date' column
df = dataframes[0]
for df_key in dataframes[1:]:
    df = pd.merge(df, df_key, on='date', how='outer', sort=)

# # Sort the DataFrame by 'date'
# df.sort_values(by='date', inplace=True)
# df.reset_index(drop=True, inplace=True)

print(df)


ABT
            date                                1INCH AAVE  ACH  ADX AGLD  \
0     1620172800  6.198889182909350886505987800422162    0  NaN  NaN  NaN   
1     1620259200                                    0    0  NaN  NaN  NaN   
2     1620345600                                    0    0  NaN  NaN  NaN   
3     1620345600                                    0    0  NaN  NaN  NaN   
4     1620432000  7.501175367472296865262179965221419    0  NaN  NaN  NaN   
...          ...                                  ...  ...  ...  ...  ...   
6384  1590192000                                  NaN  NaN  NaN  NaN  NaN   
6385  1590278400                                  NaN  NaN  NaN  NaN  NaN   
6386  1590364800                                  NaN  NaN  NaN  NaN  NaN   
6387  1590451200                                  NaN  NaN  NaN  NaN  NaN   
6388  1590537600                                  NaN  NaN  NaN  NaN  NaN   

     AIOZ ALCX                                 ALEPH  ALI  ... WAMPL  \

In [297]:
pd.merge(dataframes[0], dataframes[2], on='date', how='outer')

,date,1INCH,ACH
0,2021-05-05,6.198889182909350886505987800422162,NaN
1,2021-05-06,0,NaN
2,2021-05-07,0,NaN
3,2021-05-08,7.501175367472296865262179965221419,NaN
4,2021-05-09,7.013277210932177249030120005274198,NaN
...,...,...,...
717,2023-04-22,0.499615126015460299085202419788568,0.03415661559966581037354978924419563
718,2023-04-23,0.4920020605819506923451426390583383,0.03258253758008225400505158217573351
719,2023-04-24,0.4887931085038128001571252921271883,0.03339610474467720369572257032154693
720,2023-04-25,0.4882807108156256261262625488251618,0.03324787206427607103105338068331978


In [307]:
import pandas as pd
from datetime import datetime

def unix_to_datetime(unix_time):
    return datetime.utcfromtimestamp(unix_time).strftime('%Y-%m-%d')

dataframes = []

# Extract date and priceUSD from the data for each key and create a DataFrame for each key
for key in full_data.keys():
    if full_data[key]:
        try:
            extracted_data = [{'date': unix_to_datetime(item['date']), key: float(item['priceUSD'])} for item in full_data[key]]
            df_key = pd.DataFrame(extracted_data).groupby('date').mean().reset_index().set_index('date')
            dataframes.append(df_key)
        except:
            print(key)

# Concatenate all the DataFrames using the 'date' column as index
df = pd.concat(dataframes, axis=1)

# Reset the index to make 'date' a regular column again
df.reset_index(inplace=True)

# Sort the DataFrame by 'date'
df.sort_values(by='date', inplace=True)
df.reset_index(drop=True, inplace=True)

print(df)


ABT
           date     1INCH       AAVE       ACH       ADX  AGLD      AIOZ  \
0    2020-05-18       NaN        NaN       NaN       NaN   NaN       NaN   
1    2020-05-19       NaN        NaN       NaN       NaN   NaN       NaN   
2    2020-05-20       NaN        NaN       NaN       NaN   NaN       NaN   
3    2020-05-21       NaN        NaN       NaN       NaN   NaN       NaN   
4    2020-05-22       NaN        NaN       NaN       NaN   NaN       NaN   
..          ...       ...        ...       ...       ...   ...       ...   
963  2023-04-22  0.499615  70.533725  0.034157  0.188395   NaN  0.024867   
964  2023-04-23  0.492002  69.937280  0.032583  0.182199   NaN  0.024904   
965  2023-04-24  0.488793  69.442760  0.033396  0.180009   NaN  0.023522   
966  2023-04-25  0.488281  71.051235  0.033248  0.185380   NaN  0.022590   
967  2023-04-26  0.501417  72.163764  0.035028  0.188622   NaN  0.023156   

     ALCX     ALEPH       ALI  ...     WAMPL          WBTC         WETH  \
0     Na

/var/folders/6p/njxvnljx3b36_1d566nq5mjh0000gn/T/ipykernel_41403/3421497949.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.reset_index(inplace=True)


In [311]:
df.to_csv('data/tokenPrices.csv', index=False)

In [361]:
# row 245 first time there is more than 1 value
len(df.iloc[245].dropna().values[1:])

12

In [310]:
full_data['AGLD'][-1]

{'date': 1681603200,
 'priceUSD': '0.5569612414199895143517048155118745',
 'token': {'name': 'Adventure Gold',
  'poolCount': '0',
  'id': '0x32353a6c91143bfd6c7d363b546e62a9a2489a20',
  'symbol': 'AGLD',
  'totalSupply': '28240',
  'totalValueLocked': '1724150.150496338649480509',
  'totalValueLockedUSD': '960284.8082149025245473043367214897',
  'txCount': '40464',
  'volume': '338490267.16456769946960734',
  'volumeUSD': '1108969998.872881391047323207237449'}}

In [306]:
dataframes[0].groupby('date')

In [301]:
full_data['AGLD']

[{'date': 1630540800,
  'priceUSD': '2.807884357682617796153079220815433',
  'token': {'name': 'Adventure Gold',
   'poolCount': '0',
   'id': '0x32353a6c91143bfd6c7d363b546e62a9a2489a20',
   'symbol': 'AGLD',
   'totalSupply': '28240',
   'totalValueLocked': '1724150.150496338649480509',
   'totalValueLockedUSD': '960284.8082149025245473043367214897',
   'txCount': '40464',
   'volume': '338490267.16456769946960734',
   'volumeUSD': '1108969998.872881391047323207237449'}},
 {'date': 1630627200,
  'priceUSD': '4.575812054297896009771808181679488',
  'token': {'name': 'Adventure Gold',
   'poolCount': '0',
   'id': '0x32353a6c91143bfd6c7d363b546e62a9a2489a20',
   'symbol': 'AGLD',
   'totalSupply': '28240',
   'totalValueLocked': '1724150.150496338649480509',
   'totalValueLockedUSD': '960284.8082149025245473043367214897',
   'txCount': '40464',
   'volume': '338490267.16456769946960734',
   'volumeUSD': '1108969998.872881391047323207237449'}},
 {'date': 1630713600,
  'priceUSD': '3.751

In [293]:
dataframes[2]

,date,ACH
0,2021-07-09,0
1,2021-07-10,0
2,2021-07-18,0
3,2021-07-21,0
4,2021-07-28,0
...,...,...
628,2023-04-22,0.03415661559966581037354978924419563
629,2023-04-23,0.03258253758008225400505158217573351
630,2023-04-24,0.03339610474467720369572257032154693
631,2023-04-25,0.03324787206427607103105338068331978


In [ ]:
# add empty values to the list to make it the same length as the dataframe
sources += [''] * (len(df) - len(sources))
df['Sources'] = sources

---

import pandas as pd, 

-----

In [252]:


stats = {'name': ['AERGO', '1INCH'], 'values': [0, 5]}

In [256]:
len(test_tokenData['AERGO'])

0

In [181]:
# test code
# test_tokenData = {}
# test_tokenData['1INCH'] = ""
# test_tokenData['AAVE'] = ""
# test_tokenData['ABT'] = ""

In [269]:
test_tokenData = fetch_data(test_tokenData)

In [231]:
import json
with open('data/tokenPriceData.json', 'w') as fp:
    json.dump(test_tokenData, fp, indent=4) 

In [232]:
tokenData = test_tokenData.copy()

In [248]:
test_tokenData[list(tokenData)[list(tokenData).index('C98')]]

In [247]:
list(tokenData)[list(tokenData).index('C98')]

'C98'

In [191]:
tokenData['1INCH'][-1]['date'] - tokenData['ABT'][-1]['date']

86400

In [ ]:
tokenData = test_tokenData.copy()

In [271]:
len(test_tokenData)

239

In [272]:

for t in list(tokenData):

    # check if no data or if most recent value is older than 1 day
    if not tokenData[t]:
        print('query', t)

query AERGO
query ATA
query AUCTION
query C98
query CLV
query COVAL
query CRPT
query CTX
query CVC
query DDX
query DEXT
query DNT
query DREP
query DYP
query ERN
query FARM
query FIS
query FOX
query GHST
query HIGH
query IDEX
query KRL
query LOKA
query METIS
query MULTI
query MV
query REP
query STX
query UNFI
query UPI
query WCFG


In [184]:
for t in list(test_tokenData):
    if not test_tokenData[t]:
        print(t)
        break

AERGO


In [172]:
len(test_tokenData['1INCH'])

722

In [147]:
import time
t= 'AAVE'
if not test_tokenData[t] or (test_tokenData[t][-1]['date'] < (time.time() - (60 * 60 * 24))):
    print('empty')

In [121]:
import time
for i in [1, 2]:
    print(i)
    time.sleep(3)

1
2


In [141]:
test_tokenData['AAVE'][1]['date']

1682467200

In [142]:
list(test_tokenData)

['1INCH',
 'AAVE',
 'ABT',
 'ACH',
 'ADX',
 'AERGO',
 'AGLD',
 'AIOZ',
 'ALCX',
 'ALEPH',
 'ALI',
 'ALICE',
 'ALPHA',
 'AMP',
 'ANKR',
 'ANT',
 'APE',
 'API3',
 'ARB',
 'ARPA',
 'ASH',
 'ASM',
 'AST',
 'ATA',
 'AUCTION',
 'AUDIO',
 'AVT',
 'AXL',
 'AXS',
 'BADGER',
 'BAL',
 'BAND',
 'BAT',
 'BICO',
 'BIT',
 'BLUR',
 'BLZ',
 'BNT',
 'BOBA',
 'BOND',
 'BTRST',
 'BUSD',
 'C98',
 'cbETH',
 'CELR',
 'CHR',
 'CHZ',
 'CLV',
 'COMP',
 'COTI',
 'COVAL',
 'CQT',
 'CRO',
 'CRPT',
 'CRV',
 'CTSI',
 'CTX',
 'CUBE',
 'CVC',
 'CVX',
 'DAI',
 'DAR',
 'DDX',
 'DENT',
 'DEXT',
 'DIA',
 'DNT',
 'DPI',
 'DREP',
 'DYDX',
 'DYP',
 'ELA',
 'ELON',
 'ENJ',
 'ENS',
 'ERN',
 'EUL',
 'EUROC',
 'FARM',
 'FET',
 'FIS',
 'FORT',
 'FORTH',
 'FOX',
 'FRAX',
 'FTM',
 'FX',
 'FXS',
 'GAL',
 'GALA',
 'GFI',
 'GHST',
 'GLM',
 'GNO',
 'GODS',
 'GRT',
 'GTC',
 'GUSD',
 'GYEN',
 'HFT',
 'HIGH',
 'HOPR',
 'IDEX',
 'ILV',
 'IMX',
 'INDEX',
 'INJ',
 'INV',
 'IOTX',
 'JAM',
 'JASMY',
 'JUP',
 'KEEP',
 'KEY',
 'KNC',
 'KP3R',
 '

In [140]:
if test_tokenData['AAVE'][1]['date'] < (time.time() - (60 * 60 * 24)): 
    print('Out of date')

In [133]:
time.time() - (60 * 60 * 24)

1682426184.965666

In [105]:
test_tokenData = fetch_data(test_tokenData)

{'data': {'tokenDayDatas': [{'date': 1620172800, 'priceUSD': '0', 'token': {'name': 'Aave Token', 'poolCount': '0', 'id': '0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9', 'symbol': 'AAVE', 'totalSupply': '18240', 'totalValueLocked': '48463.36325879649889409', 'totalValueLockedUSD': '3497298.708587004739853102626657427', 'txCount': '69093', 'volume': '5278285.550181781990239253', 'volumeUSD': '775045587.3797846095564087604214378'}}, {'date': 1620259200, 'priceUSD': '0', 'token': {'name': 'Aave Token', 'poolCount': '0', 'id': '0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9', 'symbol': 'AAVE', 'totalSupply': '18240', 'totalValueLocked': '48463.36325879649889409', 'totalValueLockedUSD': '3497298.708587004739853102626657427', 'txCount': '69093', 'volume': '5278285.550181781990239253', 'volumeUSD': '775045587.3797846095564087604214378'}}, {'date': 1620345600, 'priceUSD': '0', 'token': {'name': 'Aave Token', 'poolCount': '0', 'id': '0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9', 'symbol': 'AAVE', 'tot

In [127]:
test_tokenData['AAVE'] = result['data']['tokenDayDatas']

In [118]:
test_tokenData[list(tokens)[1]] = ""
test_tokenData[list(tokens)[1]]
len(test_tokenData)

2

In [119]:
test_tokenData[list(tokens)[1]] = result['data']['tokenDayDatas']
test_tokenData[list(tokens)[1]]
len(test_tokenData)

2

In [83]:
list(tokens)[0]

'1INCH'

In [86]:
test_tokenData[list(tokens)[0]] = result['data']['tokenDayDatas']

In [88]:
test_tokenData[list(tokens)[0]]

[{'date': 1682380800,
  'priceUSD': '71.05123543457388696721563256560407',
  'token': {'name': 'Aave Token',
   'poolCount': '0',
   'id': '0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9',
   'totalValueLocked': '48463.36325879649889409',
   'symbol': 'AAVE',
   'totalSupply': '18240',
   'totalValueLockedUSD': '3497298.708587004739853102626657427',
   'txCount': '69093',
   'volume': '5278285.550181781990239253',
   'volumeUSD': '775045587.3797846095564087604214378'}},
 {'date': 1682467200,
  'priceUSD': '72.16376399448951305875792399079857',
  'token': {'name': 'Aave Token',
   'poolCount': '0',
   'id': '0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9',
   'totalValueLocked': '48463.36325879649889409',
   'symbol': 'AAVE',
   'totalSupply': '18240',
   'totalValueLockedUSD': '3497298.708587004739853102626657427',
   'txCount': '69093',
   'volume': '5278285.550181781990239253',
   'volumeUSD': '775045587.3797846095564087604214378'}}]